In [162]:
using CSV, DataFrames

In [163]:
primAuthor = "Postolache"
fileOwnPapers = "PP_JL_savedrecs_mine.txt"
fileCitations = "PP_JL_savedrecs.txt"

"PP_JL_savedrecs.txt"

In [164]:
fileOwnCSV = "PP_JL_MINE_test.csv"
fileOwnTXT = "PP_JL_total.txt"
fileOwnUAIC = "PP_JL_UAIC1.txt"

"PP_JL_UAIC1.txt"

In [165]:
#root = dirname(@__FILE__)
root = dirname("/mnt/e/Stoleriu/Doc/CV/auto/")
inputAIS = joinpath(root, "AI_factor_full_1997-2021.txt")
inputISI = joinpath(root, "IF_factor_full_1997-2021.txt")
AIFirstYear = 1997
AILastYear = 2021

2021

Pentru calculele pe 5 ani (sau orce alta durata):

In [166]:
ANini = 1997
ANfin = 2022

2022

In [167]:
AIS = CSV.read(inputAIS, DataFrame; header=true, delim='\t')
AIS[8157, [1, 2, 3, 14, 25, 26]]

,Journal,1997,1998,2009,2020,2021
,String31,Float64,Float64,Float64,Float64,Float64
8157,IEEE MAGN LETT,0.789,0.789,0.789,0.409,0.409


In [168]:
ISI = CSV.read(inputISI, DataFrame; header=true, delim='\t')
ISI[8157, [1, 2, 3, 14, 25, 26]]

,Journal,1997,1998,2009,2020,2021
,String31,Float64,Float64,Float64,Float64,Float64
8157,IEEE MAGN LETT,1.692,1.692,1.692,1.549,1.549


In [169]:
#root = dirname(@__FILE__)
#root = dirname("/mnt/e/Stoleriu/Doc/CV/auto/")
inputMINE = joinpath(root, fileOwnPapers)
inputCIT = joinpath(root, fileCitations)

"/mnt/e/Stoleriu/Doc/CV/auto/PP_JL_savedrecs.txt"

In [170]:
MINEraw = CSV.read(inputMINE, DataFrame; header=true, delim='\t')
sort!(MINEraw, [:PY], rev = true)
eltype.(eachcol(MINEraw));

┌ Warning: thread = 1 warning: parsed expected 69 columns, but didn't reach end of line around data row: 1. Parsing extra columns and widening final columnset
└ @ CSV /home/lali/.julia/packages/CSV/9LsxT/src/file.jl:635


In [171]:
#MINE = MINEraw[:, [2, 9, 24, 30, 32, 42, 45, 46, 47, 52, 53, 54, 55]];
#MINE = MINEraw[:, [2, 9, 24, 32, 42, 45, 46, 47, 52, 53, 54, 55]];  # fara citari (col 30)
MINE = MINEraw[:, [2, 9, 25, 32, 43, 46, 47, 53, 54, 55, 56]];  # fara citari (col 30)
eltype.(eachcol(MINE));

In [172]:
#MINE[2, :PY] = 2021   # ADV MAT no year!!!!
eltype.(eachcol(MINE));

In [173]:
#VSCodeServer.vscodedisplay(MINE)

Pregatim MINE cu noi coloane

In [174]:
rename!(MINE,:"J9" => :"Journal")
insertcols!(MINE, :NA => 0);
insertcols!(MINE, :NEFF => 0.0);
insertcols!(MINE, :PRIM => 0);
insertcols!(MINE, :AI => 0.0);
insertcols!(MINE, :AINEFF => 0.0);
insertcols!(MINE, :IF => 0.0);
insertcols!(MINE, :IFNA => 0.0);
insertcols!(MINE, :TIMESC => 0);
insertcols!(MINE, :CT => 0.0);
### 5 ani
insertcols!(MINE, :PRIM5 => 0);
insertcols!(MINE, :AINEFF5 => 0.0);
insertcols!(MINE, :IFNA5 => 0.0);
insertcols!(MINE, :TIMESC5 => 0);
insertcols!(MINE, :CT5 => 0.0);
### UAIC
insertcols!(MINE, :UAIC1 => 0.0);
insertcols!(MINE, :UAIC9 => 0.0);

In [175]:
#VSCodeServer.vscodedisplay(MINE)

Gasim numarul de autori si prim autor

In [176]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    MINE[row_index, :NA] = (1+count(";", lucrare[:AU]))

    if MINE[row_index, :NA] <= 5
        MINE[row_index, :NEFF] = MINE[row_index, :NA]
    elseif MINE[row_index, :NA] <= 15
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 5.0) / 2.0
    elseif MINE[row_index, :NA] <= 75
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 15.0) / 3.0
    else
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 45.0) / 4.0
    end

    #println( titlecase(MINE[row_index, :AU]))
    #println(findfirst(primAuthor, titlecase(MINE[row_index, :AU]))[1])
    if findfirst(primAuthor, titlecase(MINE[row_index, :AU]))[1] == 1
        MINE[row_index, :PRIM] = 1
        if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
            MINE[row_index, :PRIM5] = 1
        end
    elseif !ismissing(lucrare[:RP])
        if occursin(primAuthor, lucrare[:RP])
            MINE[row_index, :PRIM] = 1
            if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
                MINE[row_index, :PRIM5] = 1
            end
        end
    end
    #println(row_index, MINE[row_index, [:NA]])
    #println(row_index, MINE[row_index, [:PRIM]])
end

In [177]:
#VSCodeServer.vscodedisplay(MINE)

In [178]:
#MINE[:, [6, 7, 8, 12, 14, 15, 16, 17]]
MINE.PY = Int64.(MINE.PY)
MINE[ismissing.(MINE[!, :PY]), [1, 6, 7, 8, 12, 14, 15, 16, 17]]

,AU,PY,VL,BP,NA,PRIM,AI,AINEFF,IF
,String,Int64,Int64?,Int64?,Int64,Int64,Float64,Float64,Float64


In [179]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

In [180]:
AISsubsect = filter(r -> any(occursin.(Vector(MINE[1, [:Journal]]), r.Journal)), AIS)
#subsect2 = filter(r -> (["ADV MATER"][1] .== r.Journal[1]), AIS)
size(AISsubsect, 1)
#["ADV MATER"][1] .== subsect[1, [:Journal]][1]

1

In [181]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    #lucrare = MINE[1, :]
    if(ismissing(lucrare[:Journal]))
        continue
    end
    for AISrow in eachrow(AIS)
        if( lucrare[:Journal] == AISrow[:Journal] )
            if (lucrare[:PY] >= AIFirstYear)
                anString = string(lucrare[:PY])
            else
                anString = string(AIFirstYear)
            end
            if (lucrare[:PY] > AILastYear)
                anString = string(AILastYear)
            end
            #println(AISrow[anString])
            MINE[row_index, :AI] = AISrow[anString]
            MINE[row_index, :AINEFF] = MINE[row_index, :AI] / MINE[row_index, :NEFF]
            if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
                MINE[row_index, :AINEFF5] = MINE[row_index, :AINEFF]
            end
            
        end
    end
    for ISIrow in eachrow(ISI)
        if( lucrare[:Journal] == ISIrow[:Journal] )
            if (lucrare[:PY] >= AIFirstYear)
                anString = string(lucrare[:PY])
            else
                anString = string(AIFirstYear)
            end
            if (lucrare[:PY] > AILastYear)
                anString = string(AILastYear)
            end
            #println(ISIrow[anString])
            MINE[row_index, :IF] = ISIrow[anString]
            MINE[row_index, :IFNA] = ISIrow[anString] / MINE[row_index, :NA]
            if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
                MINE[row_index, :IFNA5] = MINE[row_index, :IFNA]
                MINE[row_index, :UAIC1] = (25 + 60*MINE[row_index, :IF]) / MINE[row_index, :NA]
            end
        end
    end
end

In [182]:
#VSCodeServer.vscodedisplay(MINE)
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

In [183]:
I = sum(MINE[!, :AINEFF])

4.35763653026847

In [184]:
P = sum(MINE[!, :PRIM] .* MINE[!, :AI])

0.331

In [185]:
I5 = sum(MINE[!, :AINEFF5])

4.35763653026847

In [186]:
P5 = sum(MINE[!, :PRIM5] .* MINE[!, :AI])

0.331

In [187]:
UAIC_1_1 = sum(MINE[!, :UAIC1])

1034.1658190920512

############################
Citations
############################

In [188]:
MINECITEraw = CSV.read(inputCIT, DataFrame; header=true, delim='\t');
# autocitari
#for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITEraw))
#    if( occursin(primAuthor, lucrareCIT.AU) )
#        println(lucrareCIT.AU)
#    end
#end
eltype.(eachcol(MINECITEraw));

┌ Warning: thread = 1 warning: parsed expected 69 columns, but didn't reach end of line around data row: 1. Parsing extra columns and widening final columnset
└ @ CSV /home/lali/.julia/packages/CSV/9LsxT/src/file.jl:635


In [189]:
#MINECITEraw
#VSCodeServer.vscodedisplay(MINECITEraw)

In [190]:
# eliminare autocitari
MINECITEraw = filter(row -> !(occursin(primAuthor, row.AU)),  MINECITEraw);
for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITEraw))
    if( occursin(primAuthor, lucrareCIT.AU) )
        println(lucrareCIT.AU)
    end
end

In [191]:
#unele citari missing PY (early access)
for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITEraw))
    if( ismissing(lucrareCIT.PY) )
        lucrareCIT.PY = 2021
    end
end

In [192]:
#VSCodeServer.vscodedisplay(MINECITEraw[1,1:70])

In [193]:
MINECITE = MINECITEraw[:, [2, 9, 31, 43, 45, 46, 47, 53, 54, 55, 56]];
#MINEraw[:, [2, 9, 25, 32, 43, 46, 47, 53, 54, 55, 56]];  # fara citari (col 30)
eltype.(eachcol(MINECITE));
rename!(MINECITE,:"J9" => :"Journal");

In [194]:
#VSCodeServer.vscodedisplay(MINECITE)

In [195]:
#CITES = fill!(Vector{Vector{Int64}}(undef, size(MINE)[1]), [0]);
CITES = Vector{Int64}[];
for i in 1:size(MINE)[1]
    push!(CITES, [0])
end
CITES_IFs = Vector{Float64}[];
for i in 1:size(MINE)[1]
    push!(CITES_IFs, [0.0])
end

In [196]:
CITES5 = Vector{Int64}[];
for i in 1:size(MINE)[1]
    push!(CITES5, [0])
end
CITES5_IFs = Vector{Float64}[];
for i in 1:size(MINE)[1]
    push!(CITES5_IFs, [0.0])
end

In [197]:
for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
    calcul_cit_UAIC = 0.0
    if ( ismissing(lucrareMINE[:DI]) )
        #println("Skip paper $row_indexMINE noDOI")
        continue
    end
    #println("Analyzing paper ", lucrareMINE[:DI])
    for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITE))
        if ( occursin(lucrareMINE[:DI], lucrareCIT[:CR]) )
            #println("Paper $row_indexMINE cited by $row_indexCIT")
            ISI_value = 0.0
            push!(CITES[row_indexMINE], row_indexCIT)
            push!(CITES_IFs[row_indexMINE], ISI_value)
            if (( !ismissing(lucrareCIT[:PY]) ) && ( !ismissing(lucrareCIT[:Journal]) ))
                for ISIrow in eachrow(ISI)
                    if( lucrareCIT[:Journal] == ISIrow[:Journal] )
                        an_publicare_citare = lucrareCIT[:PY]
                        if (an_publicare_citare > AILastYear)
                            an_publicare_citare = AILastYear
                        end
                        if (an_publicare_citare < AIFirstYear)
                            an_publicare_citare = AIFirstYear
                        end
                        ISI_value = ISIrow[string(an_publicare_citare)]
                        push!(CITES_IFs[row_indexMINE], ISI_value)
                        #CITES_IFs[row_indexMINE, size(CITES_IFs[row_indexMINE])] = ISI_value
                        break
                    end
                end
                if (lucrareCIT[:PY] <= ANfin) && (lucrareCIT[:PY] >= ANini)
                    push!(CITES5[row_indexMINE], row_indexCIT)
                    push!(CITES5_IFs[row_indexMINE], ISI_value)
                    calcul_cit_UAIC = calcul_cit_UAIC + (10 + 20 * ISI_value) / lucrareMINE[:NA]
                end
            end
        end
    end
    MINE[row_indexMINE, :TIMESC] = size(CITES[row_indexMINE])[1]-1
    MINE[row_indexMINE, :CT] = MINE[row_indexMINE, :TIMESC] / MINE[row_indexMINE, :NEFF]
    MINE[row_indexMINE, :TIMESC5] = size(CITES5[row_indexMINE])[1]-1
    MINE[row_indexMINE, :CT5] = MINE[row_indexMINE, :TIMESC5] / MINE[row_indexMINE, :NEFF]
    MINE[row_indexMINE, :UAIC9] = calcul_cit_UAIC

end;

In [198]:
CITES5[2]

1-element Vector{Int64}:
 0

In [199]:
size(CITES5_IFs[2])

(1,)

In [200]:
C = sum(MINE[!, :CT])

177.88615457885615

In [201]:
C5 = sum(MINE[!, :CT5])

175.22165241435397

In [202]:
UAIC_1_9 = sum(MINE[!, :UAIC9])

10526.09268938577

In [203]:
outputMINEcsv = joinpath(root, fileOwnCSV)
CSV.write(outputMINEcsv, MINE)

"/mnt/e/Stoleriu/Doc/CV/auto/PP_JL_MINE_test.csv"

In [204]:
ENV["COLUMNS"]=500
MINE[1, filter(x -> !(x in ["CR","RP"]), names(MINE))];

In [205]:
using Printf
outputMINEtxt = joinpath(root, fileOwnUAIC)
open(outputMINEtxt, "w") do file
    for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
        if ( !ismissing(lucrareMINE.BP) )
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), pp.$(lucrareMINE.BP)-$(lucrareMINE.EP) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        else
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), art.no.$(lucrareMINE.AR) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        end
        contor = 1
        for (idx_citare, citare) in enumerate(CITES5[row_indexMINE])
            if (citare == 0)
                continue
            end
            write(file, "\t\t[$row_indexMINE.$contor] $(MINECITE[citare,:].AU); $(MINECITE[citare,:].TI); $(MINECITE[citare,:].Journal), vol.$(MINECITE[citare,:].VL) ($(MINECITE[citare,:].PY)), DOI:$(MINECITE[citare,:].DI) \n")
            write(file, "\t\t\tIF citare: $(CITES5_IFs[row_indexMINE][idx_citare])\n")
            contor = contor + 1
        end
        write(file, "nr.autori: $(lucrareMINE.NA) \t\t IF: $(lucrareMINE.IF) \t\t UAIC I.1(ISI): $(@sprintf("%8.4f", lucrareMINE.UAIC1)) \t\t nr.citari: $(lucrareMINE.TIMESC5) \t\t UAIC I.9(Citari): $(@sprintf("%9.4f", lucrareMINE.UAIC9)) \n")
        write(file, '-'^400, '\n')
    end
    write(file, '-'^400, '\n')
    write(file, '-'^400, '\n')
    write(file, "Total criteriul I.1(UAIC) = $(@sprintf("%9.4f", UAIC_1_1))\t\t\tTotal criteriul I.9(UAIC) = $(@sprintf("%9.4f", UAIC_1_9))")
end

78

In [206]:
outputMINEtxt = joinpath(root, fileOwnTXT)
open(outputMINEtxt, "w") do file
    for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
        if ( !ismissing(lucrareMINE.BP) )
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), pp.$(lucrareMINE.BP)-$(lucrareMINE.EP) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        else
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), art.no.$(lucrareMINE.AR) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        end
        contor = 1
        for (idx_citare, citare) in enumerate(CITES5[row_indexMINE])
            if (citare == 0)
                continue
            end
            write(file, "\t\t[$row_indexMINE.$contor] $(MINECITE[citare,:].AU); $(MINECITE[citare,:].TI); $(MINECITE[citare,:].Journal), vol.$(MINECITE[citare,:].VL) ($(MINECITE[citare,:].PY)), DOI:$(MINECITE[citare,:].DI) \n")
            #write(file, "\t\t\tIF citare: $(CITES5_IFs[row_indexMINE][idx_citare])\n")
            contor = contor + 1
        end
        write(file, "autori=$(lucrareMINE.NA) \t autori_eff=$(lucrareMINE.NEFF) \t AIS=$(lucrareMINE.AI) \t prim=$(@sprintf("%5.3f", lucrareMINE.PRIM)) \t inf=$(@sprintf("%5.3f", lucrareMINE.AINEFF)) \t citari=$(lucrareMINE.TIMESC) \t cit=$(@sprintf("%5.3f", lucrareMINE.CT)) \n")
        write(file, '-'^400, '\n')
    end
    write(file, '*'^120, '\n')
    write(file, "P = $(@sprintf("%9.4f", P)) \t\t\t I = $(@sprintf("%9.4f", I)) \t\t\t C = $(@sprintf("%9.4f", C)) \n")
    write(file, '*'^120, '\n')
end

121